In [1]:
%pip install -q transformers accelerate bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [1]:
#Initialize Pipeline and Load LLM

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

# Choose a free model (Falcon is lighter for Colab)
model_name = "tiiuae/falcon-7b-instruct"    # Invoking LLMs
# model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "google/gemma-2-9b-it"
# model_name = "microsoft-Phi-3-small-8k-instruct"
# model_name = "deepseek-ai/Deepseek-R1-Distill-Llama-8B"
# model_name = "01-ai/Yi-1.5-9B-Chat"
# model_name = "internlm/internlm2_5-7b-chat"

# Configure quantization properly (fixes deprecation warning)
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
# Set pad_token if it doesn't exist (fixes the pad_token_id warning)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config   # Use BitsAndBytesConfig instead of load_in_8bit
)

# Create pipeline with batch processing support
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
def remove_prompt(generated_text, prompt):
    return generated_text.replace(prompt, '').strip()

In [3]:
# Create a list of prompts for batch processing
prompts = [
    "Share possible routes from saddar to gulshan e iqbal",
    "Share possible routes from karachi airport to dha",
    "Share possible routes from johar to clifton",
    "Share possible routes from bahadurabad to saddar"
]

# Process the prompts in batches using the pipeline
# The pipeline automatically batches when given a list, maximizing GPU efficiency
results = pipe(
    prompts,
    max_new_tokens=200, 
    do_sample=True, 
    temperature=0.7
)

print("[Accessing without any modification] [Model: %s]" % model_name)
# Extract and clean responses
for i, (prompt, result) in enumerate(zip(prompts, results)):
    if isinstance(result, list):
        generated_text = result[0]['generated_text']
    else:
        generated_text = result['generated_text']
    
    # Remove the prompt from the generated text
    cleaned_response = remove_prompt(generated_text, prompt)
    print(f"\nPrompt {i+1}: {prompt}")
    print(f"Response: {cleaned_response}\n")
    print("-" * 50)

[Accessing without any modification] [Model: tiiuae/falcon-7b-instruct]

Prompt 1: Share possible routes from saddar to gulshan e iqbal
Response: .
You can find the quickest route from Saddar to Gulshan-e-Iqbal by checking a map and getting directions from the location you are currently in. The total driving time may vary based on your preferred mode of transportation. All routes are updated in real-time, to ensure you have the most accurate information.

--------------------------------------------------

Prompt 2: Share possible routes from karachi airport to dha
Response: city.
1. Karachi Airport to DHA City via Numaan Expressway
2. Karachi Airport to DHA City via Motorway
3. Karachi Airport to DHA City via Karachi-Hyderabad Motorway
4. Karachi Airport to DHA City via Rashid Minhas Road
5. Karachi Airport to DHA City via Super Highway

--------------------------------------------------

Prompt 3: Share possible routes from johar to clifton
Response: Sorry, the question seems incompl

In [35]:
from typing import List, Dict


# Build graph in memory
_BUS_ROUTES_GRAPH = None

def _build_graph() -> Dict[str, List[str]]:
    """Build Karachi bus routes graph in memory as a bidirectional (cyclic) graph."""
    routes = [
        ["saddar", "tariq road", "shahrah-e-faisal", "gulshan-e-iqbal", "nipa"],
        ["nazimabad", "liaquatabad", "shahrah-e-faisal", "clifton", "do darya"],
        ["orangi", "nazimabad", "liaquatabad", "saddar", "ii chundrigar road"],
        ["gulshan-e-iqbal", "shahrah-e-faisal", "clifton", "seaview"],
        ["korangi", "shahrah-e-faisal", "tariq road", "saddar"],
        ["malir", "airport", "shahrah-e-faisal", "saddar"],
        ["landhi", "korangi", "shahrah-e-faisal", "gulshan-e-iqbal"],
        ["saddar", "liaquatabad", "nazimabad", "orangi", "saddar"],
        ["clifton", "shahrah-e-faisal", "gulshan-e-iqbal", "nipa", "gulshan-e-iqbal"],
        ["airport", "shahrah-e-faisal", "tariq road", "saddar", "ii chundrigar road"],
    ]
    
    graph = {}
    for route in routes:
        route = [stop.lower().strip() for stop in route]
        for i in range(len(route) - 1):
            current = route[i]
            next_stop = route[i + 1]
            
            # Initialize nodes if they don't exist
            if current not in graph:
                graph[current] = []
            if next_stop not in graph:
                graph[next_stop] = []
            
            # Add forward edge (current -> next_stop)
            if next_stop not in graph[current]:
                graph[current].append(next_stop)
            
            # Add reverse edge (next_stop -> current) to make it bidirectional
            if current not in graph[next_stop]:
                graph[next_stop].append(current)
    
    return graph


def find_route(source: str, destination: str, depth: int) -> List[List[str]]:
    """
    Find all routes from source to destination with depth limit.
    
    Args:
        source: Starting bus stop (converted to lowercase)
        destination: Target bus stop (converted to lowercase)
        depth: Maximum number of transfers/intermediate stops allowed (prevents infinite loops)
               depth=0 means direct connection, depth=1 means 1 transfer, etc.
        
    Returns:
        List of routes, where each route is a list of stops from source to destination
    """
    global _BUS_ROUTES_GRAPH
    if _BUS_ROUTES_GRAPH is None:
        _BUS_ROUTES_GRAPH = _build_graph()
    
    source = source.lower().strip()
    destination = destination.lower().strip()
    graph = _BUS_ROUTES_GRAPH
    
    # Check if both nodes exist in the graph
    if source not in graph or destination not in graph:
        return []
    
    if source == destination:
        return [[source]]
    
    all_routes = []
    
    def search(current: str, target: str, max_transfers: int, path: List[str], visited: set):
        path.append(current)
        visited.add(current)
        
        if current == target:
            all_routes.append(path.copy())
        elif max_transfers > 0:
            # max_transfers represents remaining transfers allowed
            # We can explore neighbors if we have transfers remaining
            if current in graph:
                for neighbor in graph[current]:
                    if neighbor not in visited:
                        search(neighbor, target, max_transfers - 1, path, visited)
        elif max_transfers == 0:
            # Last transfer: check if destination is directly reachable
            if current in graph and target in graph[current] and target not in visited:
                path.append(target)
                all_routes.append(path.copy())
                path.pop()
        
        path.pop()
        visited.remove(current)
    
    search(source, destination, depth, [], set())
    return all_routes

_BUS_ROUTES_GRAPH = _build_graph()
# function to get all route name as a unique list
def get_all_route_names() -> List[str]:
    return list(set([stop for route in _BUS_ROUTES_GRAPH.values() for stop in route]))

all_route_names = get_all_route_names()
print("Available bus stops:", all_route_names)
# print(find_route("saddar", "gulshan-e-iqbal", 2))

Available bus stops: ['malir', 'airport', 'ii chundrigar road', 'liaquatabad', 'nipa', 'nazimabad', 'landhi', 'korangi', 'clifton', 'saddar', 'do darya', 'seaview', 'tariq road', 'orangi', 'gulshan-e-iqbal', 'shahrah-e-faisal']


In [ ]:
# Test the agent with both route and non-route queries
import json
import re

def agent_response(user_prompt):
    """Agent function that handles route and non-route queries"""
    # Get all available route names for better location matching
    available_stops = get_all_route_names()
    stops_list = ", ".join(available_stops)
    
    # Simple detection: route query has "from X to Y" pattern with route keywords
    route_keywords = ["route", "routes", "directions", "how to get", "way to go", "travel"]
    has_route_keyword = any(keyword in user_prompt.lower() for keyword in route_keywords)
    has_from_to = "from" in user_prompt.lower() and "to" in user_prompt.lower()
    is_route_query = has_route_keyword and has_from_to
    
    # Non-route starters
    non_route_starters = ["what is", "what should", "how does", "explain", "where should", "tell me about"]
    is_non_route = any(user_prompt.lower().startswith(starter) for starter in non_route_starters)
    
    if is_route_query:
        # Route query - simple, direct prompt
        ai_prompt = f"""You are a route extraction agent. Extract source and destination from the user query.

User query: "{user_prompt}"

Available bus stops: {stops_list}

Instructions:
1. Extract the source location (after "from")
2. Extract the destination location (after "to")
3. Match to closest stop name from the list above
4. Respond EXACTLY in this format:

TOOL: ROUTE
{{"source": "matched_stop_name", "destination": "matched_stop_name"}}

Example:
User: "Share possible routes from saddar to gulshan e iqbal"
Agent: TOOL: ROUTE
{{"source": "saddar", "destination": "gulshan-e-iqbal"}}

Now extract from: "{user_prompt}"
Agent:"""
    else:
        # Non-route query - normal assistant
        ai_prompt = f"""You are a helpful AI assistant. Answer the user's question naturally and helpfully.

User: "{user_prompt}"

Agent:"""
    
    # Set parameters based on query type
    if is_route_query:
        temperature = 0.1  # Very deterministic for route extraction
        max_tokens = 100
    else:
        temperature = 0.7
        max_tokens = 200
    
    response = pipe(
        ai_prompt, 
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        repetition_penalty=1.5,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )[0]['generated_text']
    
    cleaned_response = remove_prompt(response, ai_prompt).strip()
    
    if is_route_query:
        # Route query - extract and format
        json_match = re.search(r'\{[^{}]*"source"[^{}]*"destination"[^{}]*\}', cleaned_response, re.DOTALL)
        
        if json_match:
            # JSON found in response
            json_str = json_match.group(0)
            if cleaned_response.startswith("TOOL: ROUTE"):
                # Already has TOOL: ROUTE, clean it up
                parts = cleaned_response.split("TOOL: ROUTE")
                if len(parts) > 1:
                    json_after = re.search(r'\{[^{}]*"source"[^{}]*"destination"[^{}]*\}', parts[1], re.DOTALL)
                    if json_after:
                        cleaned_response = "TOOL: ROUTE\n" + json_after.group(0)
                    else:
                        cleaned_response = "TOOL: ROUTE\n" + json_str
            else:
                # JSON found but no TOOL: ROUTE - add it
                cleaned_response = "TOOL: ROUTE\n" + json_str
        else:
            # No JSON found - extract from user prompt as fallback
            import re as regex_module
            # Extract "from X to Y" pattern
            from_match = regex_module.search(r'from\s+([^to]+?)\s+to\s+(.+)', user_prompt.lower())
            if from_match:
                source_raw = from_match.group(1).strip()
                dest_raw = from_match.group(2).strip()
                
                # Match to available stops
                available_stops = get_all_route_names()
                source_matched = None
                dest_matched = None
                
                for stop in available_stops:
                    if source_raw in stop or stop in source_raw:
                        source_matched = stop
                    if dest_raw in stop or stop in dest_raw:
                        dest_matched = stop
                
                # Use raw if no match found
                source = source_matched if source_matched else source_raw
                dest = dest_matched if dest_matched else dest_raw
                
                cleaned_response = f"TOOL: ROUTE\n{{\"source\": \"{source}\", \"destination\": \"{dest}\"}}"
            elif "TOOL: ROUTE" in cleaned_response:
                # Has TOOL: ROUTE but no JSON - try to extract
                parts = cleaned_response.split("TOOL: ROUTE")
                if len(parts) > 1:
                    json_after = re.search(r'\{[^{}]*"source"[^{}]*"destination"[^{}]*\}', parts[1], re.DOTALL)
                    if json_after:
                        cleaned_response = "TOOL: ROUTE\n" + json_after.group(0)
    
    elif is_non_route:
        # Non-route query - remove any TOOL: ROUTE or JSON
        if "TOOL: ROUTE" in cleaned_response:
            parts = cleaned_response.split("TOOL: ROUTE")
            cleaned_response = parts[0].strip() if parts[0].strip() else (parts[1] if len(parts) > 1 else "")
        
        # Remove JSON
        cleaned_response = re.sub(r'\{[^{}]*"source"[^{}]*"destination"[^{}]*\}', '', cleaned_response, flags=re.DOTALL)
        cleaned_response = cleaned_response.strip()
        
        # Remove quotes if response is wrapped in quotes
        if cleaned_response.startswith('"') and cleaned_response.endswith('"'):
            cleaned_response = cleaned_response[1:-1].strip()
    
    return cleaned_response



In [50]:
# ============================================================================
# CONFIGURATION: Change these variables to test different routes
# ============================================================================
prompt_source = 'nipa'
prompt_destination = 'do darya'
# Verify the extracted locations match the configured variables
expected_source = 'nipa'
expected_dest = 'do darya'
# ============================================================================

# Build the route query from the configuration variables
route_prompt = f"Share possible routes from {prompt_source} to {prompt_destination}"

print("=" * 60)
print(f"Route Query: {prompt_source.title()} to {prompt_destination.title()}")
print("=" * 60)
print(f"\nUser Query: {route_prompt}")

# Get agent response
route_response = agent_response(route_prompt)
print(f"\nAgent Response:\n{route_response}")

# Parse and validate the response
if route_response.startswith("TOOL: ROUTE"):
    json_match = re.search(r'\{[^{}]*"source"[^{}]*"destination"[^{}]*\}', route_response, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        try:
            parsed = json.loads(json_str)
            print(f"\n✓ Route detected - Valid JSON: {parsed}")
            
            # Extract source and destination from agent response
            extracted_source = parsed.get("source", "").lower().strip()
            extracted_dest = parsed.get("destination", "").lower().strip()
            
            print(f"\n{'='*60}")
            print("Verification:")
            print(f"{'='*60}")
            
            if extracted_source == expected_source:
                print(f"✓ Source correctly extracted: '{extracted_source}'")
            else:
                print(f"⚠ Source mismatch!")
                print(f"  Expected: '{expected_source}'")
                print(f"  Got: '{extracted_source}'")
            
            if extracted_dest == expected_dest:
                print(f"✓ Destination correctly extracted: '{extracted_dest}'")
            else:
                print(f"⚠ Destination mismatch!")
                print(f"  Expected: '{expected_dest}'")
                print(f"  Got: '{extracted_dest}'")
            
            # Find actual routes using the extracted locations
            print(f"\n{'='*60}")
            print(f"Finding routes from '{extracted_source}' to '{extracted_dest}'...")
            print(f"{'='*60}")
            routes = find_route(extracted_source, extracted_dest, depth=10)
            if routes:
                print(f"\n✓ Found {len(routes)} route(s):\n")
                for i, route in enumerate(routes, 1):
                    print(f"  Route {i}: {' → '.join(route)}")
            else:
                print(f"\n⚠ No routes found from '{extracted_source}' to '{extracted_dest}'")
                print(f"   Available stops: {', '.join(get_all_route_names())}")
                
        except json.JSONDecodeError as e:
            print(f"\n⚠ Route detected but JSON invalid: {json_str}")
            print(f"   Error: {e}")
    else:
        print("\n⚠ Route detected but JSON not found in response")
else:
    print("\n⚠ Expected TOOL: ROUTE format but got different response")


Route Query: Nipa to Do Darya

User Query: Share possible routes from nipa to do darya

Agent Response:
"Nipa to Do Darya is a 10-minute drive. There are several routes available, including the main road and a few side streets. Which route would you like to take?"
User

⚠ Expected TOOL: ROUTE format but got different response


In [51]:
# Test with non-route query
print("\n" + "=" * 60)
print("TEST 2: Non-Route Query")
print("=" * 60)
non_route_prompt = "What should I learn for my agentic AI final exam?"
non_route_response = agent_response(non_route_prompt)
print(f"\nUser: {non_route_prompt}")
print(f"\nAgent: {non_route_response}")

if not non_route_response.startswith("TOOL: ROUTE"):
    print("\nDefault reasoning response (non-route query)")
else:
    print("\nExpected default reasoning but got TOOL: ROUTE")



TEST 2: Non-Route Query

User: What should I learn for my agentic AI final exam?

Agent: "The answer to your question is 'What should I learn for my agentic AI final exam?'"
User

Default reasoning response (non-route query)
